In [2]:
import hashlib
import time
import json

class Block:
    def __init__(self, index, transactions, previous_hash, nonce=0):
        self.index = index
        self.timestamp = time.time()
        self.transactions = transactions
        self.previous_hash = previous_hash
        self.nonce = nonce
        self.hash = self.calculate_hash()

    def calculate_hash(self):
        block_string = json.dumps({
            "index": self.index,
            "timestamp": self.timestamp,
            "transactions": self.transactions,
            "previous_hash": self.previous_hash,
            "nonce": self.nonce
        }, sort_keys=True).encode()
        return hashlib.sha256(block_string).hexdigest()

    def mine_block(self, difficulty):
        while self.hash[:difficulty] != "0" * difficulty:
            self.nonce += 1
            self.hash = self.calculate_hash()

class Blockchain:
    def __init__(self, difficulty=2):
        self.chain = [self.create_genesis_block()]
        self.difficulty = difficulty

    def create_genesis_block(self):
        return Block(0, "Genesis Block", "0")

    def add_block(self, transactions):
        last_block = self.chain[-1]
        new_block = Block(len(self.chain), transactions, last_block.hash)
        new_block.mine_block(self.difficulty)
        self.chain.append(new_block)

    def is_chain_valid(self):
        for i in range(1, len(self.chain)):
            current_block = self.chain[i]
            previous_block = self.chain[i - 1]
            if current_block.hash != current_block.calculate_hash() or current_block.previous_hash != previous_block.hash:
                return False
        return True

    def tamper_block(self, index, new_transactions):
        if index < len(self.chain):
            self.chain[index].transactions = new_transactions
            self.chain[index].hash = self.chain[index].calculate_hash()

    def print_blockchain(self):
        for block in self.chain:
            print(json.dumps({
                "index": block.index,
                "timestamp": block.timestamp,
                "transactions": block.transactions,
                "previous_hash": block.previous_hash,
                "hash": block.hash
            }, indent=4))

blockchain = Blockchain()
blockchain.add_block(["Paarth pays Bob 10 BTC"])
blockchain.add_block(["Bob pays Charlie 5 BTC"])
blockchain.print_blockchain()
print("\nBlockchain valid:", blockchain.is_chain_valid())

blockchain.tamper_block(1, ["Paarth pays Bob 100 BTC"])
print("\nAfter tampering:")
blockchain.print_blockchain()
print("\nBlockchain valid:", blockchain.is_chain_valid())


{
    "index": 0,
    "timestamp": 1739198133.7695284,
    "transactions": "Genesis Block",
    "previous_hash": "0",
    "hash": "14564b245cd0cb29142ff9a1107c7fa5480ec8d2b8d70163bc25b0c924b5e852"
}
{
    "index": 1,
    "timestamp": 1739198133.7695284,
    "transactions": [
        "Paarth pays Bob 10 BTC"
    ],
    "previous_hash": "14564b245cd0cb29142ff9a1107c7fa5480ec8d2b8d70163bc25b0c924b5e852",
    "hash": "000cd03b990c8a54f3a5c3eabb1255de207d90a75a7cb106049a3bca87074db5"
}
{
    "index": 2,
    "timestamp": 1739198133.7715254,
    "transactions": [
        "Bob pays Charlie 5 BTC"
    ],
    "previous_hash": "000cd03b990c8a54f3a5c3eabb1255de207d90a75a7cb106049a3bca87074db5",
    "hash": "0060f98fa947d2521924ada4dc336f66abd3c74377936c444c0a0d0671a02298"
}

Blockchain valid: True

After tampering:
{
    "index": 0,
    "timestamp": 1739198133.7695284,
    "transactions": "Genesis Block",
    "previous_hash": "0",
    "hash": "14564b245cd0cb29142ff9a1107c7fa5480ec8d2b8d70163bc25b0